# MLPy — Sesión 3: Red neuronal desde cero (sin `nn.Module`)

**Objetivo:** implementar una red pequeña (2 capas) con PyTorch usando:
- parámetros explícitos (`W1`, `b1`, `W2`, `b2`)
- forward pass manual
- pérdida (cross-entropy) como escalar
- backward con autograd
- actualización manual tipo SGD

**Restricción:** no usar `torch.nn.Module` ni `torch.optim`.

> Pregunta guía: ¿puedes explicar cada línea del entrenamiento con conceptos de álgebra lineal y gradientes?


In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

torch.manual_seed(0)


## 1) Dataset sintético en 2D

Creamos dos nubes de puntos (clase 0 y clase 1) para visualizar y entrenar fácilmente.


In [ ]:
N = 400
x0 = torch.randn(N//2, 2) + torch.tensor([-2.0, 0.0])
x1 = torch.randn(N//2, 2) + torch.tensor([ 2.0, 0.0])

X = torch.cat([x0, x1], dim=0)
y = torch.cat([torch.zeros(N//2), torch.ones(N//2)]).long()

ds = TensorDataset(X, y)
train_loader = DataLoader(ds, batch_size=32, shuffle=True)

X.shape, y.shape

## 2) Definir parámetros (pesos y sesgos) explícitamente

Red de 2 capas:
- Entrada: 2
- Oculta: 16
- Salida: 2 (logits para 2 clases)

**Nota:** `requires_grad=True` le dice a Autograd que debe calcular gradientes para esos tensores.


In [ ]:
d_in = 2
h = 16
d_out = 2

# Inicialización simple (pequeña) para estabilidad
W1 = 0.1 * torch.randn(d_in, h, requires_grad=True)
b1 = torch.zeros(h, requires_grad=True)

W2 = 0.1 * torch.randn(h, d_out, requires_grad=True)
b2 = torch.zeros(d_out, requires_grad=True)

W1.shape, b1.shape, W2.shape, b2.shape

## 3) Forward pass manual

- Capa 1: `H = ReLU(X @ W1 + b1)`
- Capa 2: `logits = H @ W2 + b2`

Trabajamos con **logits** y usamos `cross_entropy` directamente (incluye softmax internamente).


In [ ]:
def forward(X):
    z1 = X @ W1 + b1          # (batch, h)
    h1 = torch.relu(z1)       # (batch, h)
    logits = h1 @ W2 + b2     # (batch, 2)
    return logits

# Prueba rápida
logits = forward(X[:5])
logits.shape

## 4) Pérdida (cross-entropy)

`F.cross_entropy(logits, y)` devuelve un **escalar**.
Ese escalar es el “punto de partida” para que Autograd calcule gradientes de todos los parámetros.


In [ ]:
loss = F.cross_entropy(forward(X[:32]), y[:32])
loss, loss.shape

## 5) Un paso de entrenamiento: backward + update manual (SGD)

Pasos:
1. Forward → logits
2. Loss → escalar
3. `loss.backward()` llena `W1.grad`, `b1.grad`, ...
4. Actualización manual con learning rate
5. Reiniciar gradientes a cero

**Importante:** los gradientes se acumulan; por eso se limpian cada iteración.


In [ ]:
lr = 0.1

def zero_grads():
    for p in (W1, b1, W2, b2):
        if p.grad is not None:
            p.grad.zero_()

# Un mini-batch
xb, yb = next(iter(train_loader))

zero_grads()
logits = forward(xb)
loss = F.cross_entropy(logits, yb)
loss.backward()

# Magnitudes de gradiente (diagnóstico rápido)
(W1.grad.abs().mean().item(), W2.grad.abs().mean().item(), loss.item())

In [ ]:
print("requires_grad:", W1.requires_grad)
print("is_leaf:", W1.is_leaf)
print("grad_fn:", W1.grad_fn)


In [ ]:
with torch.no_grad():
    for p in (W1, b1, W2, b2):
        p -= lr * p.grad

# Si no limpiamos, el siguiente backward acumularía gradientes
zero_grads()

"updated!" 

## 6) Entrenamiento completo (varias épocas)

Entrenamos por unas épocas y registramos la pérdida promedio.

Luego evaluamos accuracy sobre el mismo dataset (por simplicidad, aquí no usamos validación todavía).


In [ ]:
def accuracy(X, y):
    with torch.no_grad():
        logits = forward(X)
        preds = logits.argmax(dim=1)
        return (preds == y).float().mean().item()

epochs = 30
lr = 0.1

loss_history = []

for epoch in range(1, epochs + 1):
    running = 0.0
    n_batches = 0

    for xb, yb in train_loader:
        zero_grads()
        logits = forward(xb)
        loss = F.cross_entropy(logits, yb)
        loss.backward()

        with torch.no_grad():
            for p in (W1, b1, W2, b2):
                p -= lr * p.grad

        running += loss.item()
        n_batches += 1

    avg_loss = running / n_batches
    loss_history.append(avg_loss)

    if epoch in {1, 5, 10, 20, 30}:
        print(f"epoch {epoch:>2} | loss={avg_loss:.4f} | acc={accuracy(X,y):.3f}")

## 7) Visualizar la curva de pérdida

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(loss_history)
plt.xlabel("Epoch")
plt.ylabel("Average training loss")
plt.title("Training loss (manual SGD)")
plt.show()

## 8) (Opcional) Cross-entropy “a mano” (para entender)

`F.cross_entropy` combina `log_softmax` + negative log-likelihood.
Verificamos que coincide (aprox.) con el cálculo manual.


In [ ]:
def cross_entropy_manual(logits, y):
    log_probs = logits - logits.logsumexp(dim=1, keepdim=True)   # log_softmax
    nll = -log_probs[torch.arange(y.shape[0]), y]                # pick correct class
    return nll.mean()

xb, yb = next(iter(train_loader))
logits = forward(xb)

ce_torch = F.cross_entropy(logits, yb)
ce_manual = cross_entropy_manual(logits, yb)

ce_torch.item(), ce_manual.item()

## 9) Ejercicios

1. **Activation swap**: cambia `ReLU` por `tanh` y compara la curva de pérdida.  
2. **Hidden size**: prueba `h=4`, `h=64` y observa convergencia y accuracy.  
3. **Learning rate pathology**:
   - prueba `lr=1.0` (¿diverge?)
   - prueba `lr=0.001` (¿aprende lento?)
4. **Zero grads removal**: comenta `zero_grads()` y observa qué ocurre.

> Meta: justificar el comportamiento con gradientes, escala y estabilidad numérica.
